In [1]:
import openmm as mm
import shlex
import subprocess
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

import pdbfixer
import openmm.app as app
import simtk.unit as unit
from openmmforcefields.generators import GAFFTemplateGenerator as gen
from openff.toolkit import Topology, Molecule, ForceField


In [2]:
# with open('/scratch/htc/fsafarov/structures/8ef5_only_p_fv.pdb', 'r') as file_, open('/scratch/htc/fsafarov/structures/8ef5_fv_only_p_fixed.pdb', 'w') as output:
#     for line in file_:
#         if line.startswith('ATOM') and 'SOD' in line[17:20]:
#             line = line[:76] + 'Na' + line[78:]
#         if line.startswith('ATOM') and 'CLA' in line[17:20]:
#             line = line[:76] + 'Cl' + line[78:]
#         output.write(line)

In [3]:
# input_path = '/scratch/htc/fsafarov/structures/8ef5_only_p_fv.pdb'
# output_path = '/scratch/htc/fsafarov/structures/fixed.pdb'

# def patch_element(line, element):
#     line = line.rstrip('\n')
#     # Ensure line is at least 78 characters long
#     if len(line) < 78:
#         line = line.ljust(78)
#     return line[:76] + f"{element:>2}" + line[78:] + '\n'

# with open(input_path, 'r') as infile, open(output_path, 'w') as outfile:
#     for line in infile:
#         if line.startswith(('ATOM', 'HETATM')) and 'SOD' in line[17:20]:
#             line = patch_element(line, 'Na')
#         if line.startswith(('ATOM', 'HETATM')) and 'CLA' in line[17:20]:
#             line = patch_element(line, 'Cl')
#         outfile.write(line)

In [4]:
#forcefield.getUnmatchedResidues(model.topology)

In [5]:
# for bond in model.topology.bonds():
#     atom1, atom2 = bond
#     print(f"{atom1.residue.name}:{atom1.name} - {atom2.residue.name}:{atom2.name}")

In [6]:
# for res in model.topology.residues():
#     print(res.index, res.name)

#print([res for res in forcefield._templates])

#forcefield.getMatchingTemplates(model.topology)


# from openmm.app import Topology
# top = model.topology
# chains = list(top.chains())
# if len(chains) == 0:
#     new_top = Topology()
#     chain = new_top.addChain()
#     for res in top.residues():
#         new_res = new_top.addResidue(res.name, chain)
#         for atom in res.atoms():
#             new_top.addAtom(atom.name, atom.element, new_res)
#     model.topology = new_top


#forcefield.getUnmatchedResidues(model.topology)



In [7]:
forcefield = app.ForceField("charmm36.xml") 
#forcefield = app.ForceField("amber14-all.xml", "amber14/lipid17.xml", "amber14/tip3p.xml")

In [8]:
#struct = app.PDBFile('/scratch/htc/fsafarov/structures/8ef5_fv_only_p.pdb')

#pdb_x = app.PDBxFile('/scratch/htc/fsafarov/structures/8ef_wo_ion_wo_ligand.cif')

struct = pdbfixer.PDBFixer('/scratch/htc/fsafarov/structures/8ef_wo_ion_wo_ligand.cif')
struct.findMissingResidues() 
struct.findMissingAtoms()
# struct.findNonstandardResidues()
# struct.replaceNonstandardResidues()
struct.addMissingAtoms()
struct.addMissingHydrogens()



model = app.Modeller(struct.topology, struct.positions)


In [9]:
# ligand = Molecule.from_file('/scratch/htc/fsafarov/structures/8ef5/8ef5_opm_ligand.sdf')

# l_top = ligand.to_topology()

# model.add(l_top.to_openmm(), l_top.get_positions().to_openmm())


# gaff2 = gen([ligand])

# forcefield.registerTemplateGenerator(gaff2.generator)

In [10]:
system = forcefield.createSystem(model.topology)
integrator = mm.LangevinIntegrator(300*unit.kelvin, 1/unit.picosecond, 0.002*unit.picoseconds)
simulation = app.Simulation(model.topology, system, integrator)
simulation.context.setPositions(model.positions)

ValueError: No template found for residue 1 (POPC).  The set of atoms matches POPC, but the bonds are different.  Perhaps the chain is missing a terminal group?  For more information, see https://github.com/openmm/openmm/wiki/Frequently-Asked-Questions#template

In [ ]:
simulation.minimizeEnergy()

In [ ]:
integrator = mm.VerletIntegrator(2.0 * unit.femtoseconds)
integrator.setConstraintTolerance(0.00001)

1. Add ions separately and apply a force field on them, then merge, then create system (like done on ligands)
2. Problem is in createSystem (create system without ions, and only with ions, then merge)?
3. 